In [93]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [94]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 3
TOTAL_BUDGET = 1000

In [95]:
current_team_names = {"\u0110or\u0111e Petrovi\u0107",
                      "Alisson Becker",
                      "Matty Cash",
                      "Marcos Senesi Bar\u00f3n",
                      "Adrien Truffert",
                      "Virgil van Dijk",
                      "Dan Burn",
                      "Yankuba Minteh",
                      "Jaidon Anthony",
                      "Idrissa Gana Gueye",
                      "Alex Iwobi",
                      "Bruno Borges Fernandes",
                      "Erling Haaland",
                      "Jarrod Bowen",
                      "Dominic Calvert-Lewin",
                      }

In [96]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [97]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [98]:
print(all_player_data)

     clean_sheets  cost  expected_goals  form  gameweek  ict_index   id  \
0             3.0    56            0.00   3.0         7       14.2    1   
1             0.0    43            0.00   0.0         7        0.0    2   
2             0.0    40            0.00   0.0         7        0.0    3   
3             0.0    40            0.00   0.0         7        0.0    4   
4             3.0    62            0.48   6.5         7       24.1    5   
..            ...   ...             ...   ...       ...        ...  ...   
737           0.0    45            0.00   0.0         7        0.0  738   
738           0.0    45            0.00   0.0         7        0.0  739   
739           0.0    40            0.00   0.0         7        0.0  740   
740           0.0    45            0.00   0.0         7        0.0  741   
741           0.0    45            0.00   0.0         7        0.0  742   

                             name opposing_team  play_percent  \
0               David Raya Martín 

In [99]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [100]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [101]:
current_team_cost = current_team.getTotalCost()
current_team_cost

951

In [102]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [103]:
selected_team = Team.fromDataFrame(selected_team_df)

In [104]:
display(selected_team)

In [105]:
new_players = selected_team - current_team
print(new_players)


Total Score: 95.904

Goalkeepers:
- Robin Roefs	Score: 8.16	Cost: 45	Fixture Difficulty: 0.0
- Dean Henderson	Score: 3.91	Cost: 50	Fixture Difficulty: 0.0
Defenders:
- Marc Guéhi	Score: 7.46	Cost: 48	Fixture Difficulty: 0.0
- Nordi Mukiele	Score: 7.81	Cost: 40	Fixture Difficulty: 0.0
- Gabriel dos Santos Magalhães	Score: 8.55	Cost: 62	Fixture Difficulty: 0.0
- Omar Alderete (Captain) 	Score: 13.31	Cost: 40	Fixture Difficulty: 0.0
Attackers:
- Igor Thiago Nascimento Rodrigues (Vice Captain) 	Score: 8.88	Cost: 60	Fixture Difficulty: 0.0
- Danny Welbeck	Score: 7.89	Cost: 63	Fixture Difficulty: 0.0
Midfielders:
- Ismaïla Sarr	Score: 7.32	Cost: 64	Fixture Difficulty: 0.0
- Declan Rice	Score: 7.45	Cost: 65	Fixture Difficulty: 0.0
- Antoine Semenyo	Score: 7.58	Cost: 78	Fixture Difficulty: 0.0
- Anton Stach	Score: 7.59	Cost: 50	Fixture Difficulty: 0.0


In [106]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -37


In [107]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [108]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(1.7878115253204565, 1.7878115253204565)

In [109]:
START_SAMPLE_GAMEWEEK = config.CURRENT_GAMEWEEK
END_SAMPLE_GAMEWEEK = min(config.CURRENT_GAMEWEEK+2, 39)

In [110]:
matrix = FixtureDifficultyMatrix(scale_factor, START_SAMPLE_GAMEWEEK, END_SAMPLE_GAMEWEEK, config.CURRENT_SEASON)
matrix.precomputeFixtureDifficulty(scale_factor)

new_players.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [111]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [112]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 50.66499999999999


In [113]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [114]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [115]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [116]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [117]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    transferData.updateFixtureDifficulties(pMatrix)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [118]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")

Best transfer:
Transfer from Dan Burn -> Omar Alderete:
Old player: Dan Burn	Score: 3.16	Cost: 50	Fixture Difficulty: 0.46638561530098865
New player: Omar Alderete (Captain) 	Score: 13.31	Cost: 40	Fixture Difficulty: -0.8939057626602283
- Cost change: -10
- Score change: 10.15


'New total score: 84.0105'